In [1]:
import numpy as np
import pandas as pd
import os
os.chdir('/Users/chentian/research_MIT/docx files')

### An example of extracting texts from  well-structured .docx files

Since .docx file is more well-structured than txt, which preserves all the segmentations of the pdf file and easier to extract infromation from, firstly we convert the pdf into .docx file using https://smallpdf.com/pdf-to-word to get a file named 'documents-converted.docx'. Converting pdf to docx progrmmingly is laborious, and I'm still working on it if it's easier to do so.

In [2]:
#docx2txt can also extract text from links, headers and footers. 
import docx2txt
my_text = docx2txt.process('Filings (250) - 29-Jun-19 12-02-12-am-2909-3517-converted.docx')

In [3]:
#writing each paragragh into a string in a list
content = []
for line in my_text.splitlines():
    #This will ignore empty/blank lines. 
    if line != '':
    #Append to list
        content.append(line)

In [4]:
content

['Aurora Mobile Ltd. (JG) Q1 2019 Earnings Call',
 'Corrected Transcript',
 '06-Jun-2019',
 'Max Ventures & Industries Ltd. (539940.IN) Q4 2019 Earnings Call',
 'Corrected Transcript',
 '11-Jun-2019',
 'Max Ventures & Industries Ltd. (539940.IN) Q4 2019 Earnings Call',
 'Corrected Transcript',
 '11-Jun-2019',
 'Max Ventures & Industries Ltd. (539940.IN) Q4 2019 Earnings Call',
 'Corrected Transcript',
 '11-Jun-2019',
 'Max Ventures & Industries Ltd. (539940.IN) Q4 2019 Earnings Call',
 'Corrected Transcript',
 '11-Jun-2019',
 'Max Ventures & Industries Ltd. (539940.IN) Q4 2019 Earnings Call',
 'Corrected Transcript',
 '11-Jun-2019',
 'Max Ventures & Industries Ltd. (539940.IN) Q4 2019 Earnings Call',
 'Corrected Transcript',
 '11-Jun-2019',
 'Max Ventures & Industries Ltd. (539940.IN) Q4 2019 Earnings Call',
 'Corrected Transcript',
 '11-Jun-2019',
 'Max Ventures & Industries Ltd. (539940.IN) Q4 2019 Earnings Call',
 'Corrected Transcript',
 '11-Jun-2019',
 'Systemair AB (SYSR.SE) Q4 2

###### Get the information of corporate participants

To get the company, name, title and date of call of corporate participants, relevant sentences are located first by finding the position of the string 'CORPORATE PARTICIPANTS'. Then the specific information can be reached in nearby lines. <br /> In the case of rule-based recognition, corrected and raw transcript are separated for their different structures. Notice that the formatted report is just a reconstruction of the corrected transcript and doesn't contain any new information, we simply neglect the formatted report.

In [5]:
from collections import defaultdict
import re 
df_corporate=pd.DataFrame(None,  columns=['company','title', 'date of call', 'ticker'] )

def get_corporate_pre(content):
    d = defaultdict(list)
    for i in np.arange(len(content)):
        #loop for every company
        if content[i] in [' Formatted Report', 'Formatted Report']:
            break
        if content[i]=='CORPORATE PARTICIPANTS':
            j=i
            if content[i-5] in [' Corrected Transcript', 'Corrected Transcript'] or content[i-6] in [' Corrected Transcript', 'Corrected Transcript']:
                while content[j+1]!='......................................................................................................................................................................................................................................................':
                    name=content[j+1]
                    x = content[j+2].split(", ")
                    #adding the company name
                    #identify the company name after removing the characters in a parentheses and the parentheses 
                    company_raw=re.sub(r'\([^)]*\)', '', x[-1])
                    if company_raw not in ['Inc.', 'Ltd.', 'Inc. ', 'Ltd. ', 'LLC', 'LLC ']:
                        company=x[-1]
                    #adding the person's title
                        title=', '.join(x[:-1])
                    if company_raw in ['Inc.', 'Ltd.', 'Inc. ', 'Ltd. ', 'LLC', 'LLC ']:
                        company=', '.join(x[-2:])
                        title=', '.join(x[:-2])
                    d[name].append(company)
                    d[name].append(title)
                    #adding the date of call and company tickers including cases when the names of the company occupy two instead of regularly one row
                    if content[i-5]  in [' Corrected Transcript', 'Corrected Transcript']:                  
                        date=content[i-4]
                        d[name].append(date)
                        s=content[i-3]
                        ticker_list=re.findall('\(.*?\)',s)
                        if ticker_list==[]:
                            ticker=' '
                        else:
                            st=ticker_list[-1]
                            ticker=st[st.find("(")+1:st.rfind(")")]
                            d[name].append(ticker)
                    if content[i-6]  in [' Corrected Transcript', 'Corrected Transcript']: 
                        date=content[i-5]
                        d[name].append(date)
                        s=content[i-3]
                        ticker_list=re.findall('\(.*?\)',s)
                        if ticker_list==[]:
                            ticker=' '
                        else:
                            st=ticker_list[-1]
                            ticker=st[st.find("(")+1:st.rfind(")")]
                            d[name].append(ticker)
            
                    j+=2
            elif content[i-5] =='Raw Transcript' or content[i-6]=='Raw Transcript':
                while content[j+1]!='......................................................................................................................................................................................................................................................':
                    name=content[j+1]
                    title=content[j+2]
                    
                    if content[i-5]=='Raw Transcript': 
                        raw=content[j-3]
                        # remove the characters in a parentheses and the parentheses
                        string1=re.sub(r'\([^)]*\)', '', raw)
                        company=string1.rstrip()
                        d[name].append(company)
                        d[name].append(title)
                        date=content[i-4]
                        d[name].append(date)
                        s=content[i-3]
                        ticker_list=re.findall('\(.*?\)',s)
                        if ticker_list==[]:
                            ticker=' '
                        else:
                            st=ticker_list[-1]
                            ticker=st[st.find("(")+1:st.rfind(")")]
                            d[name].append(ticker)
                    if content[i-6]=='Raw Transcript': 
                        raw=content[j-4]
                        # remove the characters in a parentheses and the parentheses
                        string1=re.sub(r'\([^)]*\)', '', raw)
                        company=string1.rstrip()
                        d[name].append(company)
                        d[name].append(title)
                        date=content[i-5]
                        d[name].append(date)
                        s=content[i-3]
                        ticker_list=re.findall('\(.*?\)',s)
                        if ticker_list==[]:
                            ticker=' '
                        else:
                            st=ticker_list[-1]
                            ticker=st[st.find("(")+1:st.rfind(")")]
                            d[name].append(ticker)
                    j+=2
            else:
                for s in np.arange(j-10, j)[::-1]:
                    if content[s] in [' Corrected Transcript', 'Corrected Transcript']:
                        while content[j+1]!='......................................................................................................................................................................................................................................................':
                            name=content[j+1]
                            x = content[j+2].split(", ")
                            #adding the company name
                            #identify the company name after removing the characters in a parentheses and the parentheses 
                            company_raw=re.sub(r'\([^)]*\)', '', x[-1])
                            if company_raw not in ['Inc.', 'Ltd.', 'Inc. ', 'Ltd. ', 'LLC', 'LLC ']:
                                company=x[-1]
                            #adding the person's title
                                title=', '.join(x[:-1])
                            if company_raw in ['Inc.', 'Ltd.', 'Inc. ', 'Ltd. ', 'LLC', 'LLC ']:
                                company=', '.join(x[-2:])
                                title=', '.join(x[:-2])
                            d[name].append(company)
                            d[name].append(title)
                            date=content[s+1]
                            d[name].append(date)
                            s=content[s+2]
                            ticker_list=re.findall('\(.*?\)',s)
                            if ticker_list==[]:
                                ticker=' '
                            else:
                                st=ticker_list[-1]
                            ticker=st[st.find("(")+1:st.rfind(")")]
                            d[name].append(ticker)
                            j+=2
                        break
                
    #Although each dictionary d may contains more than 3 columns, we only use the first three.
    #This is because only one conferene call documented in one or several documents for the same corporate participant on the same day will appeared in a single pdf.
    for key in d.keys():
        if len(d[key])>4:
            d[key]=d[key][0:4]
    df1=pd.DataFrame.from_dict(d, orient='index', columns=['company','title', 'date of call', 'ticker'])
    return [df1, d.keys()]

                


SInce for some of the exceptional cases, like when the 'Corrected Transcript' exists in some irregullar places, the tickers of corresponding firms can't be found in our perceived way, we instead use the following function with the adaption that we don't manage to get those tickers.

In [38]:
from collections import defaultdict
import re 
df_corporate=pd.DataFrame(None,  columns=['company','title', 'date of call','ticker'])

def get_corporate(content):
    d = defaultdict(list)
    for i in np.arange(len(content)):
        #loop for every company
        if content[i] in [' Formatted Report', 'Formatted Report']:
            break
        if content[i]=='CORPORATE PARTICIPANTS':
            j=i
            if content[i-5] in [' Corrected Transcript', 'Corrected Transcript'] or content[i-6] in [' Corrected Transcript', 'Corrected Transcript']:
                while content[j+1]!='......................................................................................................................................................................................................................................................':
                    name=content[j+1]
                    x = content[j+2].split(", ")
                    #adding the company name
                    #identify the company name after removing the characters in a parentheses and the parentheses 
                    company_raw=re.sub(r'\([^)]*\)', '', x[-1])
                    if company_raw not in ['Inc.', 'Ltd.', 'Inc. ', 'Ltd. ', 'LLC', 'LLC ']:
                        company=x[-1]
                    #adding the person's title
                        title=', '.join(x[:-1])
                    if company_raw in ['Inc.', 'Ltd.', 'Inc. ', 'Ltd. ', 'LLC', 'LLC ']:
                        company=', '.join(x[-2:])
                        title=', '.join(x[:-2])
                    d[name].append(company)
                    d[name].append(title)
                    #adding the date of call including cases when the names of the company occupy two instead of regularly one row
                    if content[i-5]  in [' Corrected Transcript', 'Corrected Transcript']:                  
                        date=content[i-4]
                        d[name].append(date)
                        s=content[i-3]
                        ticker_list=re.findall('\(.*?\)',s)
                        if ticker_list==[]:
                            ticker=' '
                        else:
                            st=ticker_list[-1]
                            ticker=st[st.find("(")+1:st.rfind(")")]
                            d[name].append(ticker)
                    if content[i-6]  in [' Corrected Transcript', 'Corrected Transcript']: 
                        date=content[i-5]
                        d[name].append(date)
                        s=content[i-3]
                        ticker_list=re.findall('\(.*?\)',s)
                        if ticker_list==[]:
                            ticker=' '
                        else:
                            st=ticker_list[-1]
                            ticker=st[st.find("(")+1:st.rfind(")")]
                            d[name].append(ticker)
            
                    j+=2
            elif content[i-5] =='Raw Transcript' or content[i-6]=='Raw Transcript':
                while content[j+1]!='......................................................................................................................................................................................................................................................':
                    name=content[j+1]
                    title=content[j+2]
                    
                    if content[i-5]=='Raw Transcript': 
                        raw=content[j-3]
                        # remove the characters in a parentheses and the parentheses
                        string1=re.sub(r'\([^)]*\)', '', raw)
                        company=string1.rstrip()
                        d[name].append(company)
                        d[name].append(title)
                        date=content[i-4]
                        d[name].append(date)
                        s=content[i-3]
                        ticker_list=re.findall('\(.*?\)',s)
                        if ticker_list==[]:
                            ticker=' '
                        else:
                            st=ticker_list[-1]
                            ticker=st[st.find("(")+1:st.rfind(")")]
                            d[name].append(ticker)
                    if content[i-6]=='Raw Transcript': 
                        raw=content[j-4]
                        # remove the characters in a parentheses and the parentheses
                        string1=re.sub(r'\([^)]*\)', '', raw)
                        company=string1.rstrip()
                        d[name].append(company)
                        d[name].append(title)
                        date=content[i-5]
                        d[name].append(date)
                        s=content[i-3]
                        ticker_list=re.findall('\(.*?\)',s)
                        if ticker_list==[]:
                            ticker=' '
                        else:
                            st=ticker_list[-1]
                            ticker=st[st.find("(")+1:st.rfind(")")]
                            d[name].append(ticker)
                    j+=2
            else:
                for s in np.arange(i-10, i):
                    if content[s] in [' Corrected Transcript', 'Corrected Transcript']:
                        while content[j+1]!='......................................................................................................................................................................................................................................................':
                            name=content[j+1]
                            x = content[j+2].split(", ")
                            #adding the company name
                            #identify the company name after removing the characters in a parentheses and the parentheses 
                            company_raw=re.sub(r'\([^)]*\)', '', x[-1])
                            if company_raw not in ['Inc.', 'Ltd.', 'Inc. ', 'Ltd. ', 'LLC', 'LLC ']:
                                company=x[-1]
                            #adding the person's title
                                title=', '.join(x[:-1])
                            if company_raw in ['Inc.', 'Ltd.', 'Inc. ', 'Ltd. ', 'LLC', 'LLC ']:
                                company=', '.join(x[-2:])
                                title=', '.join(x[:-2])
                            d[name].append(company)
                            d[name].append(title)
                            date=content[s+1]
                            d[name].append(date)
                       
                            j+=2
                        break
                
    #Although each dictionary d may contains more than 3 columns, we only use the first three.
    #This is because only one conferene call documented in one or several documents for the same corporate participant on the same day will appeared in a single pdf.
    for key in d.keys():
        if len(d[key])>4:
            d[key]=d[key][0:4]
    df1=pd.DataFrame.from_dict(d, orient='index', columns=['company','title', 'date of call','ticker'])
    return [df1, d.keys()]

                

In [39]:
get_corporate(content)[0].head()

,company,title,date of call,ticker
Christian Arnell,Christensen International,Senior Vice President,06-Jun-2019,JG
Weidong Luo,Aurora Mobile Ltd.,"Co-Founder, Chairman & Chief Executive Officer",06-Jun-2019,JG
Fei Chen,Aurora Mobile Ltd.,Co-Founder & President,06-Jun-2019,JG
Shan-Nen Bong,Aurora Mobile Ltd.,Chief Financial Officer,06-Jun-2019,JG
Aristotelis Spiliotis,GEK Terna Holding Real Estate Construction SA,Corporate Finance,10-Jun-2019,Terna Energy SA


### Get the information of other participants

Codes are similar to find the company, name, title and date of call of other participants, in which we locate lines near the string 'OTHER PARTICIPANTS'. While each analyst may appear more than once in the text, we only record its first appearance to get the dataframe.

In [40]:
from collections import defaultdict
import re 
def get_others(content):
    d_others = defaultdict(list)
    for i in np.arange(len(content)):
        #loop for every company
        if content[i]=='OTHER PARTICIPANTS':
            j=i
            for s in np.arange(i-30,i)[::-1]:
                if content[s] in [' Formatted Report', 'Formatted Report']:
                    break
                if content[s] in [' Corrected Transcript', 'Corrected Transcript']:
                    date=content[s+1]
                    while content[j+1] not in ['MANAGEMENT DISCUSSION SECTION', '......................................................................................................................................................................................................................................................']:
                        name=content[j+1]
                        x = content[j+2].split(", ")
                        #adding the company name
                        if x[-1]=='Analyst':
                            company=', '.join(x[:-1])
                            title=x[-1]
                        else:
                            company=', '.join(x[1:])
                            #adding the person's title
                            title=x[0]
                        d_others[name].append(company)
                        d_others[name].append(title)
                    #adding the date of call and including special cases when there's no corporate participants 
                        d_others[name].append(date)       
                        j+=2
    
    #delete repeated values caused by multiple speakings of the same speaker
    d_new=defaultdict(list)
    for key in d_others:
        company=d_others[key][0]
        d_new[key].append(company)
        title=d_others[key][1]
        d_new[key].append(title)
        date=d_others[key][2]
        d_new[key].append(date)
        
    df2=pd.DataFrame.from_dict(d_new, orient='index', columns=['company','title','date of call'])
    return [df2, d_others.keys()]


In [41]:
get_others(content)[0].head()

,company,title,date of call
Bill Liu,Goldman Sachs (Asia) LLC,Analyst,06-Jun-2019
Ribery Gu,Credit Suisse (Hong Kong) Ltd.,Analyst,06-Jun-2019
Katerina Zaharopoulou,Eurobank Equities Investment Firm SA,Analyst,10-Jun-2019
Argyrios Gkonis,Axia Ventures Group Ltd. (Broker),Analyst,10-Jun-2019
Arafat Saiyed,Reliance Securities Ltd.,Analyst,11-Jun-2019


## Extract the text of corporate participants

The extraction of the relevant texts is implemented by locating the separators and the participants' names in the next line. The text are separately extracted for corporate participants and other participants like analysts due to different rules of extraction. Moreover, the categorized texts are saved in correpsonding directory and sub-directory according to their companies, name and date of call.

In [ ]:
#create a list that contains all the names
name_corporate=list(d.keys())
name_answer= ["A" + name for name in name_corporate]

In [42]:
def extract_corporate_management(content):
    k=0
    for i in np.arange(len(content)):  
        if content[i]=='......................................................................................................................................................................................................................................................':
            #deaing with texts in management discussion part
            if content[i+1] in name_corporate:
                if i+3<len(content)-1:
                    j=i+3
                else:
                    break
                k+=1
                text=[]
                while content[j] not in ['......................................................................................................................................................................................................................................................' ,'Disclaimer','QUESTION AND ANSWER SECTION']:
                    text.append(content[j])
                    j+=1
                textFilename =str(k) + ".txt"
                name=content[i+1]
                filename = "/Users/chentian/research_intern/"+'text'+'/'+'corporate participants'+'/'+'management discussion section'+'/'+df1.loc[name, 'company']+'/'+name+'/'+df1.loc[name, 'date of call']+'/'+textFilename
                os.makedirs(os.path.dirname(filename), exist_ok=True)
                with open(filename, "w") as f:
                    sent= '\n'.join(text)
                    f.write(sent)
                
        

In [43]:
#dealing with texts in Q&A part；note that in raw transcript of some companies, e.g., Fabege AB, texts for other participants are nonexistent.
def extract_corporate_QandA(content):
    k=0
    paths=set()
    for s in np.arange(len(content)):
        if content[s]=='QUESTION AND ANSWER SECTION':
            for i in np.arange(s+1, len(content)):
                if content[i]=='......................................................................................................................................................................................................................................................':
                   #to get the texts of anwers in the Q&A section
                    if content[i+1] in name_answer:
                        if i+3<len(content)-1:
                            j=i+3
                        else:
                            break
                        k+=1
                        text=[]
                        while content[j] not in ['......................................................................................................................................................................................................................................................' ,'Disclaimer']:
                            text.append(content[j])
                            j+=1
                        textFilename =str(k) + ".txt"
                        #get the name after the character 'A'
                        name=content[i+1][1:]
                        filename = "/Users/chentian/research_intern/"+'text'+'/'+'corporate participants'+'/'+'Q&A section'+'/'+df1.loc[name, 'company']+'/'+name+'/'+df1.loc[name, 'date of call']+'/'+textFilename
                        #os.makedirs(os.path.dirname(filename), exist_ok=True)
                        path_name = "/Users/chentian/research_intern/"+'text'+'/'+'corporate participants'+'/'+'Q&A section'+'/'+df1.loc[name, 'company']+'/'+name+'/'+df1.loc[name, 'date of call']
                        paths.add(path_name)
                        #with open(filename, "w") as f:
                            #sent= '\n'.join(text)
                            #f.write(sent)           
                    #to get other texts besides answers in the Q&A section
                    if content[i+1] in name_corporate:
                        if i+3<len(content)-1:
                            j=i+3
                        else:
                            break
                        k+=1
                        text=[]
                        while content[j] not in ['......................................................................................................................................................................................................................................................' ,'Disclaimer']:
                            text.append(content[j])
                            j+=1
                        textFilename =str(k) + ".txt"
                        name=content[i+1]
                        filename = "/Users/chentian/research_intern/"+'text'+'/'+'corporate participants'+'/'+'Q&A section'+'/'+df1.loc[name, 'company']+'/'+name+'/'+df1.loc[name, 'date of call']+'/'+textFilename
                        #os.makedirs(os.path.dirname(filename), exist_ok=True)
                        path_name = "/Users/chentian/research_intern/"+'text'+'/'+'corporate participants'+'/'+'Q&A section'+'/'+df1.loc[name, 'company']+'/'+name+'/'+df1.loc[name, 'date of call']
                        paths.add(path_name)
                        #with open(filename, "w") as f:
                            #sent= '\n'.join(text)
                            #f.write(sent)
    return paths

## Extract the text of other participants

In [ ]:
#create a list that contains all the names
name_others=list(d_others.keys())
name_question= ["Q" + name for name in name_others]

In [44]:
def extract_others_all(content):
    k=0
    for i in np.arange(len(content)):
        if content[i]=='......................................................................................................................................................................................................................................................':
            #dealing with text in the management discussion part
            if content[i+1] in name_others:
                if i+3<len(content)-1:
                    j=i+3
                else:
                    break
                k+=1
                text=[]
                while content[j] not in ['......................................................................................................................................................................................................................................................' ,'Disclaimer','QUESTION AND ANSWER SECTION']:
                    text.append(content[j])
                    j+=1
                textFilename =str(k) + ".txt"
                name=content[i+1]
                filename = "/Users/chentian/research_intern/"+'text'+'/'+'other participants'+'/'+df2.loc[name, 'company']+'/'+name+'/'+df2.loc[name, 'date of call']+'/'+textFilename
                os.makedirs(os.path.dirname(filename), exist_ok=True)
                with open(filename, "w") as f:
                    sent= '\n'.join(text)
                    f.write(sent)        
            #dealing with text in Q&A part
            if content[i+1] in name_question:
                if i+3<len(content)-1:
                    j=i+3
                else:
                    break
                k+=1
                text=[]
                while content[j] not in ['......................................................................................................................................................................................................................................................' ,'Disclaimer']:
                    text.append(content[j])
                    j+=1
                textFilename =str(k) + ".txt"
                #get the name after the character 'Q'
                name=content[i+1][1:]
                filename = "/Users/chentian/research_intern/"+'text'+'/'+'other participants'+'/'+df2.loc[name, 'company']+'/'+name+'/'+df2.loc[name, 'date of call']+'/'+textFilename
                os.makedirs(os.path.dirname(filename), exist_ok=True)
                with open(filename, "w") as f:
                    sent= '\n'.join(text)
                    f.write(sent)


## Applying the functions above to all the docx files

In [47]:
import docx2txt
import glob
#to walk through all the .docx files
files=glob.glob("/Users/chentian/research_intern/docx files/*.docx")
#to concatenate all the dataframes of extracted information
df_corporate=pd.DataFrame(None,  columns=['company','title', 'date of call','ticker'])
df_others=pd.DataFrame(None,  columns=['company','title', 'date of call'])
all_paths=set()
for file in files:
    my_text = docx2txt.process(file)
    #writing each paragragh into a string in a list
    content = []
    for line in my_text.splitlines(): #This will ignore empty/blank lines. 
        if line != '':
            content.append(line)
    corporate=get_corporate(content)
    others=get_others(content)
    #create a list that contains all the names of corporate participants 
    name_corporate=list(corporate[1])
    name_answer= ["A" + name for name in name_corporate]
    df1=corporate[0]
    #extract_corporate_management(content)
    path=extract_corporate_QandA(content)
    all_paths.update(path)
    #create a list that contains all the names of other participants
    #name_others=list(others[1])
    #name_question= ["Q" + name for name in name_others]
    #df2=others[0]
    #extract_others_all(content)
    df_corporate=pd.concat([df_corporate, df1])
    #df_others=pd.concat([df_others, df2])


In [2]:
#all_paths

In [174]:
df_corporate.head()

,company,title,date of call,ticker
Michael L. Smith,Oglethorpe Power Corp.,President & Chief Executive Officer,10-Apr-2019,000ST4-E
Michael W. Price,Oglethorpe Power Corp.,Chief Operating Officer & Executive Vice Presi...,10-Apr-2019,000ST4-E
Elizabeth Bush Higgins,Oglethorpe Power Corp.,Chief Financial Officer & Executive Vice Presi...,10-Apr-2019,000ST4-E
Brook Bulger Taube,Sierra Income Corp.,Director,02-Apr-2019,00DBWD-E
Richard T. Allorto,Sierra Income Corp.,"Chief Financial Officer, Secretary & Treasurer",02-Apr-2019,00DBWD-E


In [97]:
df_corporate.loc['Reece Allen Kurtenbach','ticker']

'DAKT'

In [131]:
path='/Users/chentian/research_intern/text/corporate participants/Q&A section/Barnes & Noble Education, Inc./Kanuj Malhotra/25-Jun-2019'
name=path.split("/")[-2]
name

'Kanuj Malhotra'

In [132]:
(df_corporate.loc[name, 'ticker']).iloc[0]

'BNED'

In [287]:
df_others

,company,title,date of call
Juliette Liu,"Yuanta Securities Investment Consulting Co., Ltd.",Analyst,28-Mar-2019
Lincoln Kong,Goldman Sachs (Asia) LLC,Analyst,28-Mar-2019
Chen Luo,Merrill Lynch (Singapore) Pte Ltd.,Analyst,28-Mar-2019
Linda Huang,Macquarie Capital Ltd.,Analyst,28-Mar-2019
Chen Luo,Bank of America Merrill Lynch,Analyst,14-May-2019
Erwan Rambourg,"HSBC Securities USA, Inc.",Analyst,14-May-2019
Hugo Shen,Macquarie International Holdings Ltd. (China),Analyst,14-May-2019
Dustin Wei,Morgan Stanley Asia Ltd.,Analyst,14-May-2019
Mohamed Tomalieh,NCB Capital Co,Analyst,09-Apr-2019
Ashar Saleem,Saudi Kuwaiti Finance House S.S.C.,Analyst,09-Apr-2019


In [208]:
df_corporate.to_csv (r'/Users/chentian/research_intern/corporate_participants.csv', encoding='utf_8_sig') 

In [289]:
df_others.to_csv (r'/Users/chentian/research_intern/other_participants.csv', encoding='utf_8_sig') 

## Finding duplicate files in the directories

In [51]:
# dupFinder.py
import os, sys
import hashlib
 
def DelDup(parentFolder):
    # Dups in format {hash:[names]}
    dups = []
    for dirName, subdirs, fileList in os.walk(parentFolder):
        print('Scanning %s...' % dirName)
        for filename in fileList:
            # Get the path to the file
            path = os.path.join(dirName, filename)
            # Calculate hash
            file_hash = hashfile(path)
            # Add or append the file path
            if file_hash in dups:
                os.remove(path)
            else:
                dups.append(file_hash)
    return dups
  
def hashfile(path, blocksize = 65536):
    afile = open(path, 'rb')
    hasher = hashlib.md5()
    buf = afile.read(blocksize)
    while len(buf) > 0:
        hasher.update(buf)
        buf = afile.read(blocksize)
    afile.close()
    return hasher.hexdigest()
 

In [ ]:
for path in all_paths:
    DelDup(path)

In [22]:
for path in all_paths:
    if os.path.exists(path+"/result.txt"):
        os.remove(path+"/result.txt")

## Merge the text files in the same directory

In [23]:
import fileinput
import glob

for path in all_paths:
    file_list = glob.glob(path+"/*.txt")  
    with open(path+'/result.txt', 'w') as file:
        input_lines = fileinput.input(file_list)
        file.writelines(input_lines)

In [188]:
from readability import Readability
df=pd.DataFrame(None, columns=['conference call date','company name','company ticker','participant title','FKGL score', 'GF score','ARI score','CLI score','LW score','DCR score','words'])
for path in all_paths:
    d1= defaultdict(list)
    path_full=path+"/result.txt"
    FileName=path_full
    InputFile = open(FileName,'r')
    text = InputFile.read()
    r = Readability(text)
    for char in '-.,\n':
        Text=text.replace(char,' ')
    Text = Text.lower()
    words = len(Text.split())
    if words>=100:
        fk = r.flesch_kincaid()
        gf = r.gunning_fog()
        ari = r.ari()
        cl = r.coleman_liau()
        lw = r.linsear_write()
        dc = r.dale_chall()
        name=path.split("/")[-2]
        if name in df_corporate.T.columns:
            d1[name].append(path.split("/")[-1])
            d1[name].append(path.split("/")[-3])
            d1[name].append(df_corporate.loc[name, 'ticker'])
            d1[name].append(df_corporate.loc[name, 'title'])
            d1[name].append(fk.score)
            d1[name].append(gf.score)
            d1[name].append(ari.score)
            d1[name].append(cl.score)
            d1[name].append(lw.score)
            d1[name].append(dc.score)
            d1[name].append(words)
        D1=pd.DataFrame.from_dict(d1, orient='index', columns=['conference call date','company name','company ticker','participant title','FKGL score', 'GF score','ARI score','CLI score','LW score','DCR score','words'])
        df=pd.concat([df,D1])

In [ ]:
result=pd.DataFrame.from_dict(d1, orient='index', columns=['conference call date','company name','company ticker','participant title','FKGL score', 'GF score','ARI score','CLI score','LW score','DCR score','words'])
   
    

In [191]:
df

,conference call date,company name,company ticker,participant title,FKGL score,GF score,ARI score,CLI score,LW score,DCR score,words
Reece Allen Kurtenbach,29-May-2019,"Daktronics, Inc.",DAKT,"Chairman, President & Chief Executive Officer",13.409157,16.791493,14.055808,13.572761,15.844444,9.961661,969
Hubert Joly,23-May-2019,"Best Buy Co., Inc.",BBY,Chairman & Chief Executive Officer,11.847454,15.307353,12.769365,9.317176,16.763889,8.837436,939
Laurie G. Hylton,21-May-2019,Eaton Vance Corp.,EV,Chief Financial Officer and Vice President,15.778873,19.052909,17.843581,13.731534,20.772727,11.314291,1951
Steven Martin Vamos,15-May-2019,Xero Ltd.,XRO.NZ,Chief Executive Officer,12.444719,15.883957,13.623226,10.852061,16.813364,8.978859,5529
Erkan Kilimci,13-May-2019,Halkbank,HALKB.TR,Deputy General Manager,10.548685,13.474799,11.077835,10.791086,13.126582,9.575355,1567
Sandeep Mahindroo,12-Apr-2019,Infosys Ltd.,Sandeep Mahindroo INFY Sandeep Mahindroo ...,Sandeep Mahindroo Financial Controller & He...,7.856970,8.873354,8.027454,9.545844,8.468750,8.223586,238
James C. Clemmer,02-Apr-2019,"AngioDynamics, Inc.",ANGO,"President, Chief Executive Officer & Director",10.881041,14.019062,11.816724,9.431960,15.103448,7.915486,1390
Phil Urban,23-May-2019,Mitchells & Butlers Plc,MAB.GB,Chief Executive Officer & Director,10.488784,13.618833,11.176693,9.360436,14.223982,8.386857,4974
Ian Smith,15-May-2019,CYBG Plc,CYB.AU,Group Chief Financial Officer & Executive Dire...,9.773837,13.028339,10.156304,9.546231,12.824438,8.688450,7135
Wei Wen,17-May-2019,TuanChe Ltd.,"Wei Wen TC Wei Wen TC Name: ticker, dtyp...","Wei Wen Co-Founder, Chairman & Chief Execut...",14.005493,17.554162,16.740172,13.456638,19.023810,12.068885,1003


In [192]:
df.to_csv (r'/Users/chentian/research_intern/results.csv', encoding='utf_8_sig') 

In [196]:
file = open(r'/Users/chentian/research_intern/all_paths.txt','w')
file.write(str(all_paths))
file.close()